# 04-Spam-Classifier

It's time to make our first real Machine Learning application of NLP: a spam classifier!

A spam classifier is a Machine Learning model that classifier texts (email or SMS) into two categories: Spam (1) or legitimate (0).

To do that, we will reuse our knowledge: we will apply preprocessing and BOW (Bag Of Words) on a dataset of texts.
Then we will use a classifier to predict to which class belong a new email/SMS, based on the BOW.

First things first: import the needed libraries.

In [1]:
# Import NLTK and all the needed libraries
import nltk
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

Load now the dataset in *spam.csv* using pandas. Use the 'latin-1' encoding as loading option.

In [2]:
# TODO: Load the dataset 
df = pd.read_csv('spam.csv')
df.head()

,Class,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


As usual, I suggest you to explore a bit this dataset.

In [3]:
# TODO: explore the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Class    5572 non-null   object
 1   Message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


So as you see we have a column containing the labels, and a column containing the text to classify.

We will begin by doing the usual preprocessing: tokenization, punctuation removal and lemmatization.

In [20]:
# TODO: Perform preprocessing over all the text
import string
tokens = []
for e,s in enumerate(df["Message"]):
    token = nltk.word_tokenize(s)
    s = s.translate(str.maketrans('', '', string.punctuation))
    token = nltk.word_tokenize(s)
    token = [x for x in token if x not in nltk.corpus.stopwords.words('english')]
    lemma = nltk.stem.WordNetLemmatizer()
    token = [lemma.lemmatize(x) for x in token]
    tokens.append(token)
df["PMessage"] = tokens
df["PMessage"]

0       [Go, jurong, point, crazy, Available, bugis, n...
1                          [Ok, lar, Joking, wif, u, oni]
2       [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3           [U, dun, say, early, hor, U, c, already, say]
4       [Nah, I, dont, think, go, usf, life, around, t...
                              ...                        
5567    [This, 2nd, time, tried, 2, contact, u, U, �75...
5568             [Will, �, b, going, esplanade, fr, home]
5569                      [Pity, mood, Soany, suggestion]
5570    [The, guy, bitching, I, acted, like, id, inter...
5571                              [Rofl, Its, true, name]
Name: PMessage, Length: 5572, dtype: object

Ok now we have our preprocessed data. Next step is to do a BOW.

In [28]:
# TODO: compute the BOW
s = [str(x) for x in df["PMessage"]]
vectorizer = CountVectorizer()
vec = vectorizer.fit_transform(s)
vec.toarray()


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Then make a new dataframe as usual to have a visual idea of the words used and their frequencies.

In [29]:
# TODO: Make a new dataframe with the BOW
BOW = pd.DataFrame(data = vec.toarray(),columns = vectorizer.get_feature_names_out())
BOW

,008704050406,0089my,0121,01223585236,01223585334,0125698789,02,020603,0207,02070836089,...,zebra,zed,zero,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's check what is the most used word in the spam category and the non spam category.

There are two steps: first add the class to the BOW dataframe. Second, filter on a class, sum all the values and print the most frequent one.

In [30]:
# TODO: print the most used word in the spam and non spam category
s = [str(x) for x in df.loc[df['Class'] == 'spam']['Message']]
vec = vectorizer.fit_transform(s)
vec.toarray()
spamBOW = pd.DataFrame(data = vec.toarray(),columns = vectorizer.get_feature_names_out())
#spamBOW
s = [str(x) for x in df.loc[df['Class'] == 'ham']['Message']]
vec = vectorizer.fit_transform(s)
vec.toarray()
hamBOW = pd.DataFrame(data = vec.toarray(),columns = vectorizer.get_feature_names_out())
#hamBOW
print(spamBOW.sum().nlargest(5),hamBOW.sum().nlargest(5))

to      691
call    355
you     297
your    264
free    224
dtype: int64 you    1948
to     1562
the    1133
and     858
in      823
dtype: int64


You should find that the most frequent spam word is 'free', not so surprising, right?

Now we can make a classifier based on our BOW. We will use a simple logistic regression here for the example.

You're an expert, you know what to do, right? Split the data, train your model, predict and see the performance.

In [24]:
# TODO: Perform a classification to predict whether a message is a spam or not

# the way I did BOW classes doesn't support this

What precision do you get? Check by hand on some samples where it did predict well to check what could go wrong...

Try to use other models and try to improve your results.